In [10]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss,confusion_matrix,classification_report,roc_curve,auc
import pickle

In [11]:
train = pd.read_csv('train.csv')
test=pd.read_csv('test.csv')
target_col = ['toxic', 'severe_toxic', 'obscene', 'threat','insult', 'identity_hate']

In [12]:
v = TfidfVectorizer()


In [13]:
X_train = v.fit_transform(train['comment_text'])
X_test = v.transform(test['comment_text'])


In [14]:
model=[]
i=0
for label in ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']:
    y = train[label]
    model.append(LogisticRegression(solver='sag'))
    model[i].fit(X_train, y)
    i=i+1

In [15]:
y=train[target_col]
i=0
for col in target_col:
    print("Column:",col)
    pred_pro = model[i].predict_proba(X_train)[:,1]
    frp,trp,thres = roc_curve(y[col],pred_pro)
    auc_val =auc(frp,trp)
    print(auc_val)
    i=i+1

Column: toxic
0.9846255258083668
Column: severe_toxic
0.9912915485118727
Column: obscene
0.992449134045807
Column: threat
0.9955118551615831
Column: insult
0.9873653068190493
Column: identity_hate
0.9895489886453366


In [16]:
i=0
for col in target_col:
    test[col]=model[i].predict_proba(X_test)[:, 1]
    i=i+1
test

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...,0.998341,0.180688,0.994835,0.046668,0.961562,0.282676
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...,0.006599,0.001558,0.004002,0.000423,0.005719,0.003051
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap...",0.042214,0.006275,0.020677,0.001878,0.023293,0.006015
3,00017563c3f7919a,":If you have a look back at the source, the in...",0.003174,0.001640,0.003101,0.001037,0.003708,0.000931
4,00017695ad8997eb,I don't anonymously edit articles at all.,0.034268,0.003956,0.011118,0.001649,0.011520,0.003556
...,...,...,...,...,...,...,...,...
153159,fffcd0960ee309b5,". \n i totally agree, this stuff is nothing bu...",0.748311,0.004208,0.200132,0.001221,0.064731,0.008087
153160,fffd7a9a6eb32c16,== Throw from out field to home plate. == \n\n...,0.062054,0.003704,0.029493,0.002675,0.021551,0.010575
153161,fffda9e8d6fafa9e,""" \n\n == Okinotorishima categories == \n\n I ...",0.006037,0.001429,0.005584,0.000725,0.004370,0.001834
153162,fffe8f1340a79fc2,""" \n\n == """"One of the founding nations of the...",0.022918,0.001462,0.011117,0.000926,0.011423,0.008937


In [17]:
f=open("final_model.pkl", "wb")
pickle.dump(model,f)
pickle.dump(v,f)